In [1]:
import shared.config as config
import pymongo
import gridfs
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes


import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

from sqlalchemy import and_, or_

from PIL import Image, ExifTags
import magic
import io
import shared.image_util as iu
import os


In [2]:
myclient = pymongo.MongoClient(config.MONGO_URI)
stagingDb = myclient[config.DB_STAGING]
analyseDb = myclient[config.DB_ANALYSE]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [3]:
#grp_aggr = [{"$match" : {'soort': 'Artefact', '_id': ObjectId('62aa1843288d4dfe4c78f511')}}]
grp_aggr = [{"$match" : {'soort': 'Artefact', 'vondstID': {'$exists': True}}}]
df_artf = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
#df_artf.loc[df_artf['brondata'].notnull() & df_artf.brondata.str.contains('VONDST', case=False)]
#df_artf['brondata']
df_artf

""


In [4]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
filesdb = myclient[str(config.DB_FILES)]
fs = gridfs.GridFS(filesdb)



In [5]:
image = Image.open('a10fd8740268fcef460befe7a366bc49.jpg', 'r')
iu.adjustAndSaveFile('a10fd8740268fcef460befe7a366bc49.jpg', fs, analyseCol)

ObjectId('62b7633e49fd2439b0966f08')

In [ ]:
fs.get('')

In [6]:
%timeit shrinkAndSaveImage('a10fd8740268fcef460befe7a366bc49.jpg', config.IMAGE_SIZE_BIGGEST, fs)

NameError: name 'shrinkAndSaveImage' is not defined

In [ ]:
def adjustAndSaveFile(fullfilename, fs, collection):
    try:
        # First read image and make 3 versions with different sizes. And put them in a list 
        image = Image.open(fullfilename, 'r')
        mime_type = magic.from_file(fullfilename, mime=True)
        filename, file_extension = os.path.splitext(fullfilename)
        dir, filename = os.path.split(fullfilename)

        lst_images = []
        image_dict_big = {
            'image': iu.adjustImage(image, config.IMAGE_SIZE_BIGGEST),
            'size': config.IMAGE_SIZE_BIGGEST
        }
        lst_images.append(image_dict_big)
        img_med = iu.adjustImage(image, config.IMAGE_SIZE_MIDDLE, postcard=True)
        image_dict_med = {
            'image': img_med,
            'size': config.IMAGE_SIZE_MIDDLE
        }
        lst_images.append(image_dict_med)
        img_sml = img_med.copy()
        img_sml.thumbnail(config.IMAGE_SIZE_THUMB)
        image_dict_sml = {
            'image': img_sml,
            'size': config.IMAGE_SIZE_THUMB
        }
        lst_images.append(image_dict_sml)

        # Loop over all versions and store them in the filestore
        for img in lst_images:
            width = str(img['size'][0])
            height = str(img['size'][1])
        
            b = io.BytesIO()
            img['image'].save(b, "JPEG")
            b.seek(0)

            img['uuid'] = fs.put(b, content_type=mime_type, height=height, width=width, filename=filename)

        # Insert a record with metadata
        collection.insert_one({
            'fileName': filename, 'imageUUID': str(image_dict_big['uuid']), 'imageMiddleUUID': str(image_dict_med['uuid']), 'imageThumbUUID': str(image_dict_sml['uuid']),
            'fileType': file_extension.lower(), 'directory': dir, 'mime_type': mime_type 
            })  

    except Exception as err:
        msg = "Onbekende fout bij het bewaren van image, met tekst: " + str(err)
        print(msg)    
        raise Exception(msg) from err


In [ ]:
adjustAndSaveFile('a10fd8740268fcef460befe7a366bc49.jpg', fs, analyseCol)

In [ ]:
os.path.split('a10fd8740268fcef460befe7a366bc49.jpg')